In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import emcee
import corner

# Define the dataset

# Read the data from the file
file_path = './lcparam_full_long.txt'
df = pd.read_csv(file_path, delim_whitespace=True, comment='#', header=None, names=['name', 'zcmb', 'zhel', 'dz', 'mb', 'dmb', 'x1', 'dx1', 'color', 'dcolor', '3rdvar', 'd3rdvar', 'cov_m_s', 'cov_m_c', 'cov_s_c', 'set', 'ra', 'dec', 'biascor'])
#in this file I don't put the supernovae with strange names (last supernovae)


'''
x1: SALT2 light curve stretch parameter.
c: SALT2 light curve color parameter.
mb: SALT2 light curve peak magnitude in B band.
μCorr: Corrected distance modulus. 
Mass: Host galaxy stellar mass.
zcmb: CMB frame redshift.
zhel: Heliocentric redshift.
'''

# Extract the parameters we want to plot
params = df[["zcmb", "mb", "dmb", "x1","color"]].values #.values converts the name of the columns to a numpy array

# Define the log-likelihood function using the given formula
#Explanation of the likelihood:
'''
d is the data vector--> params (in the following function)
mu is the mean vector--> theta (in the following function)
sigma_inv is the inverse of the covariance matrix
'''
def log_likelihood(theta, params, Sigma_inv):
    mu = theta
    ones = np.ones(params.shape[0])
    
    diff = params - mu
    term1 = -0.5 * np.dot(np.dot(diff.T, Sigma_inv), diff)
    
    Sigma_inv_ones = np.dot(Sigma_inv, ones)
    numerator = np.dot(ones.T, np.dot(Sigma_inv, diff)) ** 2
    denominator = np.dot(ones.T, Sigma_inv_ones)
    term2 = 0.5 * (numerator / denominator)
    
    return term1 + term2

# Define the prior (assuming a uniform prior for simplicity)
def log_prior(theta):
    size_theta = len(theta)
    #create a vector of zero with the dimension fo theta
    zero_vector = np.zeros(size_theta)
    hundred_vector = 100*np.ones(size_theta)
    for i in range(size_theta):
        if np.all(theta > zero_vector) and np.all(theta < hundred_vector):
            return 0.0
    return -np.inf

# Define the posterior probability
#This is used to estimate the posterior distribution of the parameters given the data.
def log_probability(theta, params, Sigma_inv):
    lp = log_prior(theta)
    if not np.isfinite(lp):
        return -np.inf
    return lp + log_likelihood(theta, params, Sigma_inv)

# Covariance matrix and its inverse (assuming identity for simplicity, replace with actual covariance matrix)
#for the sigma I put the error of mb: dmb
Sigma = np.diag(df['dmb']**2)
Sigma_inv = np.linalg.inv(Sigma)
#print(log_likelihood([0.1, 0.1, 0.1, 0.1, 0.1], params, Sigma_inv))
#print(log_probability([0.1, 0.1, 0.1, 0.1, 0.1], params, Sigma_inv))


# Initialize the MCMC sampler
nwalkers, ndim = 50, params.shape[1] #walkers are the number of chains and ndim is the number of parameters (nwalkers must be greater or equal than ndim)
initial = params.mean(axis=0)
pos = initial + 1e-4 * np.random.randn(nwalkers, ndim)
#print(pos)
#print(1e-4 * np.random.randn(nwalkers, ndim))

sampler = emcee.EnsembleSampler(nwalkers, ndim, log_probability, args=(params, Sigma_inv))

# Run the MCMC chain
sampler.run_mcmc(pos, 5000, progress=True)
'''
# Get the samples and create the corner plot
samples = sampler.get_chain(discard=100, thin=15, flat=True)

# Create the corner plot
figure = corner.corner(samples, labels=["zcmb", "mb", "dmb"], truths=params.mean(axis=0))
plt.show()
'''


# Explanation of the code 
* Summary of the code:

We want to study the Cosmology of the Pantheon data (Redshift of the Supernovae).
To make this we will create the likelihood, then with a Montecarlo process we sample the distribution and make a corner plot to understand what we found.


* Explanation of get_mu:

With this function we can define the Cosmology model. We need this function because in the likelihood we will use mu_model (mu is the distance modulus).



* Explanation of the function relates to the likelihood:

The likelihood function measures the probability of observing the data given a set of parameters 𝜃.

The prior distribution respresents what I think about the parameters 𝜃 before observing data.

The posterior distribution represents my thought plus the observing data


In [ ]:
import sys
import platform
import os
import pandas as pd
import numpy as np
import emcee
import corner
import matplotlib.pyplot as plt

_camb_path = os.path.join(os.path.dirname(os.path.abspath('.')), 'CAMB')
sys.path.insert(0, _camb_path)

import camb
print('Using CAMB %s installed at %s' % (camb.__version__, os.path.dirname(camb.__file__)))

# Load the data
file_path = 'lcparam_full_long.txt'

# Read the first line to get the header
with open(file_path, 'r') as file:
    header_line = file.readline().strip()
    header = header_line.lstrip('#').strip()

# Remove the last word from the header (assumed to be 'biascor')
header_words = header.split()
header_words.pop()
header = ' '.join(header_words)

# Read the rest of the file into a DataFrame
data = pd.read_csv(file_path, sep='\s+', skiprows=1, header=None)
data.columns = header.split()

# Extract relevant columns
d = data['mb'].values
dmb = data['dmb'].values

# Compute the covariance matrix
Sigma = np.diag(dmb**2)
Sigma_inv = np.linalg.inv(Sigma)

print(data.head())
print("Covariance Matrix (Sigma):")
print(Sigma)
print("Inverse Covariance Matrix (Sigma_inv):")
print(Sigma_inv)

# Define the cosmological model, because later we will use it in the likelihood function
def get_mu(z, H0, Om0, w0, wa):
    pars = camb.CAMBparams()
    pars.set_dark_energy(w=w0, wa=wa)
    pars.set_cosmology(H0=H0, ombh2=0.022, omch2=Om0*0.12)
    pars.InitPower.set_params(ns=0.965)
    results = camb.get_results(pars)
    
    '''
    # Debugging: Check if w crosses -1
    z = np.array([0.1, 0.5, 1.0])  # Example redshifts
    w = w0 + wa * (z / (1 + z))
    if np.any(w > -1):
        print(f"w crosses -1 at redshifts: {z[w > -1]}")
    '''

    # Get luminosity distance in Mpc
    dl = results.comoving_radial_distance(z) * (1 + z)
    
    # Convert to distance modulus
    mu = 5 * np.log10(dl) + 25 #The constant 25 is added to adjust the units and to match the conventional definitions used in astronomy -->
    #--> for a luminosity distance of 10 Mpc, the distance modulus would be 30.
    return mu

def ln_likelihood(params, data, cov_inv):
    H0, Om0, w0, wa = params
    z = data['zcmb'].values
    d = data['mb'].values
    try:
        mu = get_mu(z, H0, Om0, w0, wa)
    except Exception as e:
        print(f"Error in get_mu: {e}")
        return -np.inf
    diff = d - mu
    term1 = -0.5 * diff @ cov_inv @ diff # @ is the matrix multiplication
    term2 = 0.5 * ((np.ones_like(diff) @ cov_inv @ diff) ** 2) / (np.ones_like(diff) @ cov_inv @ np.ones_like(diff))
    return term1 + term2

def log_prior(params):
    H0, Om0, w0, wa = params
    if not (60 < H0 < 80):
        return -np.inf
    if not (0.2 < Om0 < 0.4):
        return -np.inf
    if not (-1. < w0 < -0.):
        return -np.inf
    if not (-2. < wa < -0.5):
        return -np.inf
    
    # Constraint to avoid crossing w = -1 (relaxed)
    '''
    if w0 + wa < -1:
        return -np.inf
    '''
    return 0.0

def log_posterior(params, data, cov_inv):
    lp = log_prior(params)
    if not np.isfinite(lp):
        return -np.inf
    return lp + ln_likelihood(params, data, cov_inv)

# Initialize the MCMC sampler
initial = np.array([68, 0.3, -0.3, -1.3]) # H0=68, Om0=0.3, w0=-0.3, wa=-1.3
ndim = len(initial)
nwalkers = 500
pos = initial + 1e-4 * np.random.randn(nwalkers, ndim) # provare con 1e-1

sampler = emcee.EnsembleSampler(nwalkers, ndim, log_posterior, args=(data, Sigma_inv))

# Run the MCMC sampler
nsteps = 5000
sampler.run_mcmc(pos, nsteps, progress=True)

# Get the samples
samples = sampler.get_chain(discard=300, thin=10, flat=True)

# Plot the chains for each parameter
fig, axes = plt.subplots(ndim, figsize=(10, 7), sharex=True)
labels = ["H0", "Om0", "w0", "wa"]

for i in range(ndim):
    ax = axes[i]
    ax.plot(sampler.chain[:, :, i], color="k", alpha=0.3)
    ax.set_ylabel(labels[i])
    ax.set_xlabel("Step number")

plt.tight_layout()
plt.show()

# Plot the results
fig = corner.corner(samples, labels=["H0", "Om0", "w0", "wa"])
plt.show()


# Explanation of the corner plot

* Histograms on the diagonal:

These histograms show the marginal distribution for each parameter.

* Contour plots:
These plots represent the joint distribution between two parameters.
If the plot is elongated an diagonal, the two parameters are correlated.


* Comparison between real data and synthetic data

Now, I want to calculate the distance modulus by the the result of the corner plot.

Then I compute the theoretical distance modulus with my Cosmology model.

In this way I can compare theoretical with the observation.

In [ ]:
# My fiducial Cosmology is:
fiducial_params = {
    'H0': 68.0,
    'Om0': 0.33,
    'w0': -0.55,
    'wa': -1.3
}
z = data['zcmb'].values

# Compute theoretical magnitudes using the fiducial cosmology
mu_theoretical = get_mu(z, **fiducial_params)

# Calculate the percentiles
percentiles = [16, 50, 84]
labels = ["H0", "Om0", "w0", "wa"]

# Get the parameter estimates
estimates = np.percentile(samples, percentiles, axis=0) # 1-sigma uncertainties

# Display the results
for i, label in enumerate(labels):
    q16, q50, q84 = estimates[:, i]
    print(f"{label}: {q50:.3f} (+{q84-q50:.3f}, -{q50-q16:.3f})")

# Check the shape and some basic statistics of the samples
print("Samples shape:", samples.shape)
print("Mean of samples:", np.mean(samples, axis=0))
print("Standard deviation of samples:", np.std(samples, axis=0))

# Now I want to compute the distance modulus with this values and compare it with the theoretical distance modulus
# Compute the distance modulus using the estimated parameters
mu_estimated = get_mu(z, *estimates[1])

# Plot the distance modulus
plt.figure(figsize=(10, 6))
plt.errorbar(z, data['mb'], yerr=data['dmb'], fmt='o', label='Data')
plt.plot(z, mu_theoretical, label='Theoretical')
plt.plot(z, mu_estimated, label='Estimated')
plt.xlabel('Redshift (z)')
plt.ylabel('Distance Modulus (mu)')
plt.legend()
plt.show()



Try to understand the error of the previous code (vedere altro file: speremmu_file)